# azure_chat_agents_streaming.py - ELI5 Walkthrough
This notebook mirrors `python/samples/getting_started/workflows/agents/azure_chat_agents_streaming.py` with commentary between each step.


## Big Picture
A Writer agent drafts a slogan, the Reviewer agent critiques it, and `run_stream()` lets you watch both agents emit token-by-token updates before the final decision.


## Key Ingredients
- `WorkflowBuilder.add_agent()` turns chat agents into workflow nodes without writing custom executors.
- Streaming yields `AgentRunUpdateEvent` deltas so you can print partial generations.
- Marking the Reviewer with `output_response=True` surfaces its final reply as a workflow output.


### Workflow Diagram
```mermaid
flowchart LR
    Start(["Start Prompt"]) --> Writer[[Writer agent]]
    Writer --> Reviewer[[Reviewer agent]]
    Reviewer --> Output(["Workflow Output"])
```


### Step 1: Imports, configuration, and context
We load environment variables, import Agent Framework streaming primitives, and keep the docstring that describes the scenario.


In [ ]:
# Copyright (c) Microsoft. All rights reserved.

import asyncio

from agent_framework import AgentRunUpdateEvent, WorkflowBuilder, WorkflowOutputEvent
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from dotenv import load_dotenv
load_dotenv()

"""
Sample: Agents in a workflow with streaming

A Writer agent generates content, then a Reviewer agent critiques it.
The workflow uses streaming so you can observe incremental AgentRunUpdateEvent chunks as each agent produces tokens.

Purpose:
Show how to wire chat agents into a WorkflowBuilder pipeline using add_agent
with settings for streaming and workflow outputs.

Demonstrate:
- Automatic streaming of agent deltas via AgentRunUpdateEvent when using run_stream().
- Add an agent via WorkflowBuilder.add_agent() with output_response=True to emit final AgentRunResponse.
- Agents adapt to workflow mode: run_stream() emits incremental updates, run() emits complete responses.

Prerequisites:
- Azure OpenAI configured for AzureOpenAIChatClient with required environment variables.
- Authentication via azure-identity. Use AzureCliCredential and run az login before executing the sample.
- Basic familiarity with WorkflowBuilder, edges, events, and streaming runs.
"""




### Step 2: Build the agents and stream the workflow
`main()` creates both chat agents, wires them into a workflow, then iterates `run_stream()` to pretty-print incremental updates and the final output.


In [ ]:
async def main():
    """Build and run a simple two node agent workflow: Writer then Reviewer."""
    # Create the Azure chat client. AzureCliCredential uses your current az login.
    chat_client = AzureOpenAIChatClient(credential=AzureCliCredential())

    # Define two domain specific chat agents.
    writer_agent = chat_client.create_agent(
        instructions=(
            "You are an excellent content writer. You create new content and edit contents based on the feedback."
        ),
        name="writer_agent",
    )

    reviewer_agent = chat_client.create_agent(
        instructions=(
            "You are an excellent content reviewer."
            "Provide actionable feedback to the writer about the provided content."
            "Provide the feedback in the most concise manner possible."
        ),
        name="reviewer_agent",
    )

    # Build the workflow using the fluent builder.
    # Add agents to workflow with custom settings using add_agent.
    # Agents adapt to workflow mode: run_stream() for incremental updates, run() for complete responses.
    # Reviewer agent emits final AgentRunResponse as a workflow output.
    # Set the start node and connect an edge from writer to reviewer.
    workflow = (
        WorkflowBuilder()
        .add_agent(writer_agent, id="Writer")
        .add_agent(reviewer_agent, id="Reviewer", output_response=True)
        .set_start_executor(writer_agent)
        .add_edge(writer_agent, reviewer_agent)
        .build()
    )

    # Stream events from the workflow. We aggregate partial token updates per executor for readable output.
    last_executor_id: str | None = None

    events = workflow.run_stream("Create a slogan for a new electric SUV that is affordable and fun to drive.")
    async for event in events:
        if isinstance(event, AgentRunUpdateEvent):
            # AgentRunUpdateEvent contains incremental text deltas from the underlying agent.
            # Print a prefix when the executor changes, then append updates on the same line.
            eid = event.executor_id
            if eid != last_executor_id:
                if last_executor_id is not None:
                    print()
                print(f"{eid}:", end=" ", flush=True)
                last_executor_id = eid
            print(event.data, end="", flush=True)
        elif isinstance(event, WorkflowOutputEvent):
            print("\n===== Final output =====")
            print(event.data)

    """
    Sample Output:

    writer_agent: Charge Up Your Journey. Fun, Affordable, Electric.
    reviewer_agent: Clear message, but consider highlighting SUV specific benefits (space, versatility) for stronger
        impact. Try more vivid language to evoke excitement. Example: "Big on Space. Big on Fun. Electric for Everyone."
    ===== Final Output =====
    Clear message, but consider highlighting SUV specific benefits (space, versatility) for stronger impact. Try more
        vivid language to evoke excitement. Example: "Big on Space. Big on Fun. Electric for Everyone."
    """




### Step 3: Try it yourself
Use the helper below. In notebooks it awaits `main()` on the active loop; in scripts it falls back to `asyncio.run(main())`.


In [ ]:
import asyncio

# Helper for notebooks vs. scripts
loop = asyncio.get_event_loop()
if loop.is_running():
    # Jupyter/VS Code notebooks already have an event loop, so await directly.
    await main()
else:
    asyncio.run(main())
